# Let's get some free city data for Albuquerque, NM and make a heatmap of where in town most Breaking Bad filming took place

In [ ]:
import geopandas as gp
from pathlib import Path
import fiona
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default


In [ ]:
# data from https://www.cabq.gov/abq-data/

In [ ]:
datapath = Path('./data/geopandas/abq/')

### here we have to do a bunch of data munging - this is always a big part of any data project

In [ ]:
gdf = gp.read_file(datapath / 'abq_films.geojson')
gdf = gdf.loc[gdf.geometry != None] # there are some missing geometries, so we can skip those
gdf['Title'] = gdf['Title'].apply(lambda x:x.strip()) # let's strip off extra spaces from the Titles field
gdf.Title = ['In Plain Sight' if "Plain Sight" in i else i for i in gdf.Title]
gdf['Year'] = [i.year for i in gdf.ShootDate]
gdf = gdf.drop(columns='ShootDate') # something with this 

### what are the top 20 projects filming in ABQ?

In [ ]:
gdf.groupby('Title').count()['Type'].sort_values(ascending=False)[:20].plot.bar()

### we can just take a look at where Breaking Bad filming events took place as dots on the map

In [ ]:
gdf.loc[gdf.Title=='Breaking Bad'].explore()

### What are the other titles?

In [ ]:
gdf['Title'].unique()

In [ ]:
gdf['Type'].unique()

## read in the zone atlas - this is a grid from the city that is used for zoning, but we can use it as a base for a heatmap as it covers the city with equal-sized squares

In [ ]:
gdf_pg = gp.read_file(datapath / 'zoneatlaspagegrid.kmz')
gdf_pg.explore()

### we can assign a variable `title_to_plot` to be able to check out heatmaps of any of the titles above

In [ ]:
title_to_plot = 'Breaking Bad'

### Finally, we need to do a few things and set them to intermediate results:
- load a fresh copy of the polygons
- use `sjoin` to intersect the `gdf` points corresponding to the title named in `title_to_plot` which returns only the polygon of intersection for each point in the original point dataframe and convert to the correct CRS. Note that we can use only the columns we need (e.g. `[['Name','geometry']]) as long as `geometry` is included. This makes the results a little cleaner
- next, with that result, use `groupby` to count all the points based on their `Name` column
- count these up, choose a column from the result (we randomly chose `geometry` here) and rename the counts to a column called `points`
- finally merge the column of counts back to the original dataframe of zones noting that we will use `Name` on the left and `index` on the right

In [ ]:
gdf_pg = gp.read_file(datapath / 'zoneatlaspagegrid.kmz') # load a fresh copy of the polygons 
gdf_pg.head(3)

use `sjoin` to intersect the `gdf` points corresponding to the title named in `title_to_plot` which returns only the polygon of intersection for each point in the original point dataframe and convert to the correct CRS. Note that we can use only the columns we need (e.g. `[['Name','geometry']]) as long as `geometry` is included. This makes the results a little cleaner. 

Note we print out the lengths of the starting and ending results for reference

In [ ]:
sj_result = gdf_pg[['Name','geometry']].sjoin(
    gdf[['Title','geometry']].loc[gdf.Title==title_to_plot].to_crs(gdf_pg.crs))
print(len(sj_result), len(gdf.loc[gdf.Title==title_to_plot]), len(gdf_pg))
sj_result.head(3)

Next, with that result, use `groupby` to count all the points based on their `Name` column.

This gives us counts for each column...

In [ ]:
sj_count = sj_result.groupby(
    'Name').count()
print(len(sj_count))
sj_count.sample(4)

Count these up, choose a column from the result (we randomly chose `geometry` here)

In [ ]:
sj_sum = sj_count.geometry.rename('points')
sj_sum.head()

In [ ]:
gdf_pg = gdf_pg[['Name','geometry']].merge(sj_sum, left_on='Name', right_index=True)

### Or, we can smack _all_ that work into a single chain (WAT!?)

In [ ]:
gdf_pg = gp.read_file(datapath / 'zoneatlaspagegrid.kmz') # load a fresh copy of the polygons 
gdf_pg = gdf_pg.merge(gdf_pg[['Name','geometry']].sjoin(
    gdf[['Title','geometry']].loc[gdf.Title==title_to_plot].to_crs(gdf_pg.crs)).groupby(
    'Name').count().geometry.rename('points'), left_on='Name', right_index=True)

### finally, let's look at the result on the map overlaying the points with the heatmap. Whew!

In [ ]:
m = gdf_pg[['points','geometry']].explore(column='points', cmap='magma_r')
gdf.loc[gdf.Title==title_to_plot][['Site', 'geometry']].explore(m=m)
